In [19]:
%load_ext memory_profiler

In [11]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [26]:
from IPython.core.magic import register_cell_magic


@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

In [27]:
%%write_and_run test_func.py
def testing_func():
    x=[]
    for i in range (2):
        x+=[i]

In [28]:
 
%lprun -f testing_func testing_func()
from test_func import testing_func
%mprun -f testing_func testing_func()

Timer unit: 1e-07 s

Total time: 3.5e-06 s
File: C:\Users\Juantc93\AppData\Local\Temp\ipykernel_5084\89359751.py
Function: testing_func at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def testing_func():
     2         1          6.0      6.0     17.1      x=[]
     3         3         17.0      5.7     48.6      for i in range (2):
     4         2         12.0      6.0     34.3          x+=[i]

## Dependencies

In [4]:
import pandas as pd

## Constants

In [2]:
FILE_PATH="tweets.json.zip"

# Caso Base Pandas

In [72]:
df = pd.read_json(FILE_PATH, lines=True)\
    .assign(date=lambda x: x.date.dt.date,
            username=lambda x: x.user.apply(lambda y: y.get("username")))

df_top_ten_date=df["date"]\
    .value_counts()\
    .nlargest(10)\
    .rename_axis("date")
    

In [73]:
df_top_ten_date=df["date"]\
    .value_counts()\
    .nlargest(10)\
    .to_frame(name="twits")\
    .rename_axis("date")
    

In [215]:
df_temp=df.loc[:,["date","username"]]\
    .set_index("date")\
    .join(df_top_ten_date,how="inner")\
    .set_index("username",append=True)\
     .groupby(["date","username","twits"])\
    .agg(twits_user=pd.NamedAgg(column="twits",aggfunc="count"))\
    .assign(
        rank=lambda x: x.groupby(["date","twits"])["twits_user"].rank(axis='index',method="first",ascending=False))\
    .query("rank==1")\
    .sort_index(level="twits",ascending=False)\
    .reset_index()\
    .loc[:,["date","username"]]\
    .values

         

    
    


In [214]:
df_temp

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [190]:
df_temp.levels

AttributeError: 'DataFrame' object has no attribute 'levels'

In [22]:
df.user.apply(lambda x: x.get("username"))

0         ArjunSinghPanam
1              PrdeepNain
2          parmarmaninder
3           anmoldhaliwal
4              KotiaPreet
               ...       
117402      rickyrickstir
117403          PunjabTak
117404           ish_kayy
117405     TV9Bharatvarsh
117406          SikhVibes
Name: user, Length: 117407, dtype: object